# generate circuit for time evolution

## 描述

添加一个参数，用以指定分别负责x、y维度的qubit序号

输入是否启用shadow hamiltonian或任何其他优化

输出对应的时间演化电路

In [5]:
import numpy as np
from qiskit import QuantumCircuit, transpile, qasm2
from qiskit.circuit.library import QFT

def generate_evolution_qasm(hamiltonian_type, t, n_x, n_y, output_file="evolution.qasm"):
    """
    生成时间演化电路的QASM文件
    参数:
        hamiltonian_type: 哈密顿量类型 ('free_particle' 或 'with_potential')
        t: 演化时间
        n_x: x方向量子比特数
        n_y: y方向量子比特数
        output_file: 输出QASM文件名
    返回:
        QASM字符串并保存文件
    """
    n_total = n_x + n_y
    qc = QuantumCircuit(n_total, name=f"Evolution_t_{t}")
    
    # 核心演化函数（自由粒子部分）
    def add_free_evolution(qc, t, n_qubits, qubits):
        """添加自由粒子演化电路 (-∇²/2 部分)"""
        # QFT
        qft = QFT(num_qubits=n_qubits, inverse=False)
        qc.append(qft, qubits)
        
        # 动量空间演化: e^{-i k^2 t/2}
        # 注意：论文公式(4)的二进制权重实现
        for i in range(n_qubits):
            # 计算权重：2^{n_qubits - i - 1}
            k_weight = 2**(n_qubits - i - 1)
            # 相位旋转角度: -k^2 t/2
            angle = - (k_weight**2) * t / 2
            qc.rz(angle, qubits[i])
        
        # 逆QFT
        qft_inv = QFT(num_qubits=n_qubits, inverse=True)
        qc.append(qft_inv, qubits)
    
    # x方向演化
    add_free_evolution(qc, t, n_x, qubits=range(n_x))
    
    # y方向演化
    add_free_evolution(qc, t, n_y, qubits=range(n_x, n_x+n_y))
    
    # 势能项处理（如需）
    if hamiltonian_type == 'with_potential':
        raise NotImplementedError("势能项演化需特罗特分解，当前版本暂不支持")
        # 此处可扩展：qc.compose(add_potential_circuit(V, t), inplace=True)
    
    # 生成QASM
    qasm_str = qasm2.dumps(qc)
    
    # 保存文件
    with open(output_file, 'w') as f:
        f.write(qasm_str)
    
    return qasm_str



In [ ]:

# 示例使用 ==============================================
if __name__ == "__main__":
    # 案例1：扩张流演化 (5x5网格，自由粒子)
    qasm_output = generate_evolution_qasm(
        hamiltonian_type='free_particle',
        t=np.pi/2,  # 演化时间
        n_x=5,      # x方向5量子比特 (32点)
        n_y=5,      # y方向5量子比特 (32点)
        output_file="diverging_flow_evolution.qasm"
    )
    
    print("QASM 电路已生成！关键门统计：")
    print(f"- 总量子比特数: 10")
    print(f"- QFT/逆QFT模块: 各2个 (x/y方向)")
    print(f"- 相位旋转门(Rz): {5*2*2}个")  # 每个方向5比特 * 2次QFT * 2方向

QASM 电路已生成！关键门统计：
- 总量子比特数: 10
- QFT/逆QFT模块: 各2个 (x/y方向)
- 相位旋转门(Rz): 20个
